Preprocessing step includes filter out samples from metadata of the study
Since we are only focusing on metagenomics samples

## Install and load required libraries and setup environment


In [ ]:
library(rstudioapi)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(utils)



## Read metadata and filter out samples


In [ ]:
if(file.exists("data/hmp2_metadata.csv")){print("metadata already downloaded")}else{
fileUrl <- "https://ibdmdb.org/tunnel/products/HMP2/Metadata/hmp2_metada.csv"
utils::download.file(url, "data/hmp2_metada.csv", mode = "wb")
}
#read metadata
metaData <- read.csv("data/hmp2_metadata.csv")
#filter out samples by data type column 
metaData <- metaData[metaData$data_type== "metagenomics",]

#to see how many participants avaliable in each participant/subject in metagenomics data
groupsParticipant <- metaData %>% group_by(Participant.ID) %>%
                                  summarise(total_samples = n (),
                                  .groups = 'drop')
                                  
#samples have different visit numbers so we need to take one of them as a baseline
metaData <- metaData[metaData$visit_num == 4,]
#filter out unused columns 
metaData <- metaData [,c("External.ID","Participant.ID","diagnosis")]



## Read metagenomic data and filter out by selected samples in metadata


In [ ]:
if(file.exists("data/ecs_3.tsv"))
  {print("ecs data already exist")
}else
  {
  print("ecs data will be downloaded from URL")
  fileUrl <- "https://ibdmdb.org/tunnel/products/HMP2/WGS/1818/ecs_3.tsv.gz"
  utils::download.file(fileUrl, "data/ecs_3.tsv.gz", mode = "wb")
  R.utils::gunzip("data/ecs_3.tsv.gz")
  }

#read metagenomics data 
mgxData <- read.csv("data/ecs_3.tsv", sep = "\t")
#optional step having backup before continue
mgxData.backup <- mgxData 

#filter out gene families/rows which is not assigned and named ungrouped
mgxData <- mgxData %>% dplyr:: filter(!grepl('UNGROUPED|UNMAPPED', Feature.Sample))
#take only first row for each group of row to perform community-level pathway analysis 

#to do species-specific pathway analysis we need to skip the first row then take the rest 
#take rows  containing "|" character, so we can skip the first row 
mgxData <- mgxData %>% dplyr:: filter(grepl("\\|", Feature.Sample)) 

#split feature.Sample as gene family and others
mgxData <- mgxData %>% tidyr:: separate(Feature.Sample, c("Gene Family", "other"), "\\|")
#split others as genus and species
mgxData <- mgxData %>% tidyr:: separate(other, c("Genus", "Species"), "\\.")

#remove _g and _s from row content
mgxData$Genus <- gsub("g__","",mgxData$Genus)
mgxData$Species<- gsub("s__","",mgxData$Species)

#remove "_level4ec" from colnames of mgxData
colnames(mgxData) <- sub("_level4ec.*", "", colnames(mgxData))



## Organize metaData and metagenomic data 


In [ ]:
# store mgxData without the first three columns as tmp
tmp <- mgxData[,c(4:ncol(mgxData))]

#select samples that only exist in metadata
tmp <- tmp[names(tmp) %in% metaData$External.ID]

#there is one sample which does not exist in metadata so we need to delete it 
#see difference between two sample vectors
notExistSample <- setdiff( metaData$External.ID,colnames(tmp))

#delete not exist sample from metadata
metaData <- metaData [!(metaData$External.ID==notExistSample),]

#order both metaData and mgxData based on sample names
metaData <- metaData[order(metaData$External.ID),]
tmp <- tmp[,order(colnames (tmp))]
# Check whether sample names are in same order
all(colnames(tmp) == metaData$External.ID) 

#bind first three columns to the tmp data again 
mgxData <- cbind(mgxData[,c(1:3)],tmp  )


## Filter out gene families which has all zero values across all samples or samples across all genes


In [ ]:
#remove gene families which have all zero values across all samples then start DE analysis
BACKUP <- mgxData

#convert character data frame to a numeric data frame
tmp <- as.data.frame(sapply(mgxData[,c(4:ncol(mgxData))], as.numeric))
mgxData <- cbind(mgxData[,c(1:3)],tmp)

# assign 0 value to NA values in the data frame
mgxData[is.na(mgxData)] <- 0
#detect which rows have a total row sum >  0 
nonzero <- rowSums(mgxData[,c(4:ncol(mgxData))]) > 0
#then take only these rows
mgxData %<>% .[nonzero,]

#filter out rows which has unclassified genus column
mgxData <- mgxData %>% dplyr:: filter(!grepl("unclassified", Genus)) 



## Export preprocessed data to file


In [ ]:
write.table(metaData, "output/metaData", sep = "\t",row.names = FALSE,quote=FALSE)
write.table(mgxData, "output/mgxData", sep = "\t",row.names = FALSE,quote=FALSE )



## Creating jupyter notebook files


In [ ]:
#Jupyter Notebook file
if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
library(devtools)
library(rmd2jupyter)
rmd2jupyter("preprocessing_metagenomics.Rmd")
